In [1]:
import numpy as np
import scipy.linalg

# Matrix Diagonalization

Matrix diagonalization is another type of matrix factorization. We already saw the LU factorization when we looked at how to solve linear systems of equations.

Say we have a square matrix $A$, to diagonalize it we want to factorize it as
$$ A = PDP^{-1} $$
where $D$ is a diagonal matrix (only the diagonal elements are non-zero). Here the matrix $P$ is composed of the eigenvectors ($\vec v_i$) of $A$, and the elements of $D$ are the eigenvalues ($\lambda_i$) of $A$.

So with the matrix $P$ composed of a set of column vectors $\vec v_i$, we can also write the above expression as
$$ A\vec v_i = \lambda_i\vec v_i. $$

You might recognize this form as that also taken by the time-independent Schrödinger equation: $\mathrm{\hat H}\Psi = E\Psi$. So matrix diagonalization arises very frequently in quantum mechanical models.

We can rewrite Eq. (2) in a form like the linear systems we looked at previously
$$ (A-\lambda_i I)\vec v_i = \vec 0. $$
We want to find solutions of this where $\vec v$ is non-zero, which implies that the determinant of the matrix $(A-\lambda I)$ must be zero.

## Characteristic polynomial and eigenvalues

This bring us to the characteristic polynomial of a matrix. This is defined by the expression above:
$$
\textrm{det}(A-\lambda I) = 0.
$$

For example, if we have the matrix
$$ A = \begin{pmatrix} 2 & 1 \\ 1 & 2 \end{pmatrix}. $$
This gives us the following expression for the characteristic polynomial:
$$ \textrm{det}\begin{pmatrix} 2 - \lambda & 1 \\ 1 & 2 - \lambda \end{pmatrix} = 0. $$
We can expand this to
$$ (2-\lambda)^2 - 1 = 3 - 4 \lambda + \lambda^2 = 0. $$

If we solve this equation for $\lambda$ we'll obtain the eigenvalues of $A$. We can factor this as $(\lambda-1)(\lambda-3)$, meaning that our two eigenvalues are $\lambda=1$ and $\lambda=3$.

## Eigenvectors

Once we have obtained the eigenvalues, we can obtain the eigenvectors corresponding to each one by solving the matrix equation. For example, to find the eigenvector corresponding to the eigenvalue $\lambda = 1$ above, we need to solve
$$ \begin{pmatrix} 1 & 1 \\ 1 & 1 \end{pmatrix} \begin{pmatrix} v_1 \\ v_2 \end{pmatrix} = \begin{pmatrix} 0 \\ 0 \end{pmatrix}. $$
We can see that any $\vec v$ where $v_1 = -v_2$ would satisfy this expression. Alternatively we could say the eigenvector is any (non-zero) scalar multiple of $\begin{pmatrix} 1 \\ -1 \end{pmatrix}$.

For the eigenvalue $\lambda=3$ we need to solve
$$ \begin{pmatrix} -1 & 1 \\ 1 & -1 \end{pmatrix} \begin{pmatrix} v_1 \\ v_2 \end{pmatrix} = \begin{pmatrix} 0 \\ 0 \end{pmatrix}. $$
In this case, any $\vec v$ where $v_1 = v_2$ would satisfy this expression. So we can say the eigenvector is any scalar multiple of $\begin{pmatrix} 1 \\ 1 \end{pmatrix}$.

Note that since the eigenvector corresponding to a given eigenvalue can always be multiplied by a non-zero scalar and still be a valid eigenvector, they are often normalized such that $\left|\vec v\right| = 1$.

# Numerical approaches for calculating eigenvalues

Finding the eigenvalues from the characteristic polynomial in the example above was straightforward as this was just a $2\times2$ matrix. The more general problem of finding the eigenvalues of an $n\times n$ matrix is much more difficult, and is typically solved numerically for any moderately sized matrix. Many methods are available, each with their own advantages and disadvantages, in terms of scaling, stability, suitability for different types of matrices (e.g. sparse, banded, symmetric, real/complex, etc), and what information we want to find.

## Power Iteration

Power iteration can be used when we want to find the eigenvalue with largest absolute value in a matrix, and its associated eigenvector. These are termed the dominant eigenvalue and dominant eigenvector. This is one of the easier algorithms to follow. It consists of an iterative procedure to find the dominant eigenvalue and eigenvector of a matrix $A$:
- Pick an initial guess for the eigenvector $\vec v_0$. It must have a non-zero component along the dominant eigenvector.
- Calculate $v_1=A\vec v_0$.
- Renormalize $\vec v_1$.
- $\vec v_1$ is now a better approximation to the dominant eigenvector.
- Repeat this process, with $\vec v_{i+1} = A\vec v_i/|A\vec v_i|$ for each iteration, until a specified number of iterations have been completed or some other convergence criteria has been fulfilled.
- The eigenvalue associated with an eigenvector can be calculated using the Rayleigh quotient:
$$ \lambda_i = \frac{A \vec v \cdot \vec v}{\vec v \cdot \vec v}. $$
    - If $\vec v$ is normalized this is simplified to $\lambda = A\vec v \cdot \vec v$.

Let's write a Python function to implement this algorithm:

In [2]:
def power_iteration(a, eig_conv=0.0001, max_iter=100):
    '''
    Find the dominant eigevalue and eigenvector of a square matrix.
    
    Use the power iteration algorithm to find the dominant
    eigenvalue and eigenvector of a matrix. The eigenvalue
    is evaluted on each step using the Rayleigh quotient and
    convergence is tested against its relative change.
    
    Parameters
    ----------
    a : (N, N) array_like
        The matrix we want to analyse, in numpy format.
    eig_conv : (float, optional)
        The process is converged when the relative change in the eigenvalue
        is less than this number.
    max_iter : (int, optional)
        The maximum number of iterations before the function exits.
        
    Returns
    -------
    steps : int
        The number of steps to convergence. This would not be returned in a
        proper implementation, but it's interesting to see here how many steps
        it actually takes to converge.
    eigval : float
        The dominant eigenvalue.
    eigvec : array
        The dominant eigenvector.
        
    Raises
    ------
    LinAlgError
        If the max number of iterations are reached before convergence.
                  
    '''
    # We can start from a random guess for the eigenvector.
    eigvec = np.random.rand(a.shape[0])
    eigvec = eigvec / np.linalg.norm(eigvec)
    eigval = np.dot(np.dot(a, eigvec), eigvec)
    
    for steps in range(max_iter):
        # Calculate our new eigvec from our previous approximation.
        eigvec = np.dot(a, eigvec)
        # Renormalize it.
        eigvec = eigvec / np.linalg.norm(eigvec)
        
        # Calculate the new eigenvalue.
        eigval_new = np.dot(np.dot(a, eigvec), eigvec)
        # Test the relative change with respect to the previous approximation.
        if abs((eigval_new - eigval) / eigval) < eig_conv:
            return steps, eigval, eigvec
        # Update our approximation for the eigenvalue.
        eigval = eigval_new
        
    # If we finish the for loop without converging, raise an error.
    raise np.linalg.LinAlgError('power_iteration failed to converge to requested tolerance.')

In [3]:
power_iteration(np.array([[2, 1], [1, 2]]), eig_conv=1e-10)
# For our simple system we had previously, you can converge very accurately in a fairly small number of steps.
# Since we have a random starting point, the number of steps to convergence can vary a bit.

(9, 2.999999999817665, array([0.70710453, 0.70710903]))

In [4]:
# Lets see what we get for a large random matrix
rand_mat = np.random.rand(100, 100)
power_iteration(rand_mat, eig_conv=1e-10)

(6,
 49.88562363174823,
 array([0.0973016 , 0.10382377, 0.09595256, 0.08918906, 0.10418911,
        0.09748208, 0.11374433, 0.10291882, 0.09263224, 0.09643225,
        0.10466302, 0.1081008 , 0.10411001, 0.08958756, 0.09680659,
        0.1018447 , 0.11507118, 0.09434534, 0.09704463, 0.09784891,
        0.08997881, 0.1042847 , 0.10477967, 0.10556942, 0.09083886,
        0.09852254, 0.11171396, 0.09995302, 0.10236582, 0.10615271,
        0.10574805, 0.09560456, 0.09839899, 0.10572321, 0.10447481,
        0.09834469, 0.09555497, 0.10613201, 0.09177173, 0.10314074,
        0.09750844, 0.10919614, 0.10193561, 0.11850685, 0.10245568,
        0.09894077, 0.09294213, 0.09409628, 0.11311262, 0.09681903,
        0.10661928, 0.09398816, 0.09514876, 0.10252927, 0.09110701,
        0.1125416 , 0.10411962, 0.10069137, 0.10482574, 0.0952345 ,
        0.10659132, 0.0927492 , 0.09201186, 0.10606817, 0.08142181,
        0.09884021, 0.1011338 , 0.1014121 , 0.09557973, 0.09581024,
        0.10430032, 0.09

- Try playing around with this for random matrices of different sizes. Do you notice anything about the leading eigenvalue each time?

In practice, convergence is most slow when there is a small separation between the dominant eigenvalue and the next most dominant. This means convergence will also be slow if the dominant eigenvalue is degenerate, in which case you'll converge to the eigenvector closest your initial guess, and this may not be consistent if you start from a random guess as above.

This can be quite an efficient method for very large sparse matrices, as no explicit decomposition is needed. This is used, for example, by search engines to calculate the page rank of a website, or to show recommended users on social websites.

In [5]:
# Let's see what happens for a matrix with a degenerate dominant eigenvalue
m = np.array([[2, 1, 3], [0, 2, 3], [0, 0, 1]])
np.linalg.eig(m)

(array([2., 2., 1.]),
 array([[ 1.00000000e+00, -1.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  4.44089210e-16, -9.48683298e-01],
        [ 0.00000000e+00,  0.00000000e+00,  3.16227766e-01]]))

In [6]:
power_iteration(m, eig_conv=1e-5, max_iter=1000)

(316,
 2.0063266622373774,
 array([9.99980112e-01, 6.30683824e-03, 5.75864762e-99]))

## QR Decomposition

The approach most commonly used in many numerical libraries is based on QR decomposition. The QR decomposition, factorizes a square matrix $A$ as
$$ A = QR, $$
where $Q$ is an orthogonal matrix, meaning it's columns are orthogonal unit vectors, and $R$ is an upper triangular matrix (as was $U$ in the LU decomposition we covered previously).

The QR algorithm proceeds as follows:
- Starting from the matrix we wish to diagonalize, $A_0$, we calculate the QR decomposition, to find $Q_0$ and $R_0$.
- We then calculate $A_1$ as $R_0 Q_0$.
- The QR decomposition of $A_1$ is then found.
- This process is iterated with $A_{i+1} = R_i Q_i$
- $A_i$ will converge to an upper triangular matrix for most matrices. The diagonal elements of an upper triangular matrix equal its eigenvalues.
- This works as $A_{i+1} = R_i Q_i = Q_i^{-1}Q_i R_i Q_i = Q_i^{-1}A_i Q_i$, meaning that $A_i$ and $A_{i+1}$ are similar (have the same eigenvalues) since $Q$ is an orthogonal matrix.
- This method functions in a similar way to the power iteration method, but calculates all eigenvalues and eigenvectors simultaneously, by multiplying by a full matrix rather than a single vector.
- In practice, some additional steps are done to optimize this process, such as using so-called [Householder transformations](https://en.wikipedia.org/wiki/Householder_transformation) to convert $A$ to an upper Hessenberg form (almost upper triangular), so implementations can be somewhat involved.
- Also if you want to obtain the eigenvectors as well as eigenvalues, the product of the $Q_i$ of each step needs to be accumulated, effectively giving you what is known as the _Schur decomposition_ ($A=QUQ^{-1}$, where $Q$ is unitary and $U$ is upper triangular), which can then be used to find the eigenvectors efficiently.

## Divide and Conquer

We'll just go through a general overview of this to show you how different it is to the QR approach we discussed earlier. It's interesting how for certain types of matrices a very different approach can be more optimal then for a general matrix. In this case, the approach works out to be slightly more optimal for symmetric and hermitian matrices. It proceeds as follows:
- The matrix $M$, is first converted to an tridiagonal form $T$, using [Householder transformations](https://en.wikipedia.org/wiki/Householder_transformation).
- The tridiagonal matrix is then converted to a a block diagonal matrix plus a rank 1 correction matrix. This is the divide step, and produces the matrices from the upper and lower blocks $T_1$ and $T_2$, and the correction matrix $\beta$.
- Then there is conquer step that involves generating the diagonalization of the original matrix from the diagonalization of these submatrices.

There's more detail on this algorithm at the [wiki page](https://en.wikipedia.org/wiki/Divide-and-conquer_eigenvalue_algorithm).

# Diagonalizing Matrices in Python

There are a range of functions available, both from Numpy and SciPy to find the eigenvalues and eigenvectors of matrices. These are all interfaces to various LAPACK (a fast Fortran linear algebra library) functions, with different functions offering more optimal approaches for certain matrix types (recall banded matrices had a better scaling approach when solving linear systems). Functions to return selveral of the factorizations we've discussed are also available.

One thing to keep in mind is that the eigenvalues and eigenvectors of a real matrix can be complex, so these will usually return arrays with complex values even if your input is real, except in the case of a real symmetric or Hermitian matrix.

## Numpy functions

- [numpy.linalg.eigvals](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.eigvals.html) will compute the eigenvalues of a square array
- [numpy.linalg.eig](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.eig.html) will compute the eigenvalues and eigenvectors of a square array.
    - Both `eig()` and `eigvals()` use calls to the [geev](https://software.intel.com/en-us/mkl-developer-reference-fortran-geev) LAPACK routines for general square matrices, which uses the QR algorithm.
    - There are also versions of these functions for Hermitian or real symmetric matrices: [numpy.linalg.eigvalsh](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.eigvalsh.html) and [numpy.linalg.eigh](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.eigh.html), which use the [syevd](https://software.intel.com/en-us/mkl-developer-reference-fortran-syevd) (real symmetric), and [heevd](https://software.intel.com/en-us/mkl-developer-reference-fortran-heevd) (hermitian) LAPACK routines. Both of these use a [divide and conquer algorithm](https://en.wikipedia.org/wiki/Divide-and-conquer_eigenvalue_algorithm).
- [numpy.linalg.qr](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.qr.html) is also available for calculating the QR decomposition directly. This is also an interface to LAPACK routines.

## SciPy functions

There are many similarly available SciPy functions, which are also interfaces to the same LAPACK libraries used by their NumPy equivalents, but offer many additional parameters to tune how the calculation is done, and what is returned.

- [scipy.linalg.eigvals](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.eigvals.html) calculates eigenvalues, and [scipy.linalg.eig](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.eig.html) calculates eigenvalues and eigenvectors.
- [scipy.linalg.eigvalsh](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.eigvalsh.html) and [scipy.linalg.eigh](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.eigh.html) do the same thing for real symmetric or Hermitian matrices.
- In addition recent SciPy versions (you may need to update via pip: `sudo pip3 install scipy --upgrade`) have dedicated functions for:
    - Real symmetric or Hermitian banded matrices  [scipy.linalg.eigvals_banded](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.eigvals_banded.html) and [scipy.linalg.eig.banded](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.eig_banded.html)
    - Real symmetric tridiagonal matrices [scipy.linalg.eigh_tridiagonal](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.eigh_tridiagonal.html) and [scipy.linalg.eigh_tridiagonal](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.eigh_tridiagonal.html) 
- There is also QR decomposition with [scipy.linalg.qr](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.qr.html) and many related functions that offer more advanced options relative to the NumPy version, such as the ability to also generate a pivot matrix which orders the diagonal elements of R.

While neither NumPy or SciPy have a function specifically for a leading eigenvalue/vector as obtained from power iteration, the SciPy eigvalsh function can take an `eigvals` argument to restrict the calculation to only calculate certain eigenvalues. 

In [7]:
np.linalg.eigvals(rand_mat)

array([ 4.98856236e+01+0.j        ,  2.45762799e+00+1.66031308j,
        2.45762799e+00-1.66031308j,  2.83612563e+00+0.j        ,
       -3.03799994e+00+0.28214241j, -3.03799994e+00-0.28214241j,
        1.17344414e+00+2.59688187j,  1.17344414e+00-2.59688187j,
       -1.74336835e+00+2.35370075j, -1.74336835e+00-2.35370075j,
        1.76212341e+00+2.05604446j,  1.76212341e+00-2.05604446j,
        2.60576559e+00+0.70224999j,  2.60576559e+00-0.70224999j,
       -2.43334105e+00+1.04862719j, -2.43334105e+00-1.04862719j,
       -2.19626369e+00+1.36286264j, -2.19626369e+00-1.36286264j,
       -2.21363250e-01+2.56135291j, -2.21363250e-01-2.56135291j,
       -1.24938543e-02+2.54720771j, -1.24938543e-02-2.54720771j,
       -2.01551856e+00+1.55741701j, -2.01551856e+00-1.55741701j,
        2.58609103e+00+0.j        , -8.82181423e-01+2.34225884j,
       -8.82181423e-01-2.34225884j, -2.47630910e+00+0.j        ,
        2.00749457e+00+1.37904585j,  2.00749457e+00-1.37904585j,
        5.19946758e-01+2.

In [8]:
scipy.linalg.eigvals(rand_mat)

array([ 4.98856236e+01+0.j        ,  2.45762799e+00+1.66031308j,
        2.45762799e+00-1.66031308j,  2.83612563e+00+0.j        ,
       -3.03799994e+00+0.28214241j, -3.03799994e+00-0.28214241j,
        1.17344414e+00+2.59688187j,  1.17344414e+00-2.59688187j,
       -1.74336835e+00+2.35370075j, -1.74336835e+00-2.35370075j,
        1.76212341e+00+2.05604446j,  1.76212341e+00-2.05604446j,
        2.60576559e+00+0.70224999j,  2.60576559e+00-0.70224999j,
       -2.43334105e+00+1.04862719j, -2.43334105e+00-1.04862719j,
       -2.19626369e+00+1.36286264j, -2.19626369e+00-1.36286264j,
       -2.21363250e-01+2.56135291j, -2.21363250e-01-2.56135291j,
       -1.24938543e-02+2.54720771j, -1.24938543e-02-2.54720771j,
       -2.01551856e+00+1.55741701j, -2.01551856e+00-1.55741701j,
        2.58609103e+00+0.j        , -8.82181423e-01+2.34225884j,
       -8.82181423e-01-2.34225884j, -2.47630910e+00+0.j        ,
        2.00749457e+00+1.37904585j,  2.00749457e+00-1.37904585j,
        5.19946758e-01+2.

## Sparse Linear Algebra

As you might imagine, certain algorithms are more optimal for certain types of system. Sparse matrices (where most of the elements are zero) are special enough, and occur commonly enough, that there is an entire SciPy module for space linear algebra: [scipy.linalg.sparse](https://docs.scipy.org/doc/scipy/reference/sparse.linalg.html) separate from the main linear algebra module.

Many models can result in sparse matrices: for example finite element method solutions of partial differential equations.

The module has many linear system solvers for problems such as we saw previously, and functions to find a particular number of eigenvalues of the matrix: [scipy.sparse.linalg.eigs](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.eigs.html), or all eigenvalues/vectors: [scipy.sparse.linalg.lobpcg](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.lobpcg.html).

If you have represented your problem with a sparse matrix, these functions will be significantly faster. Also, while you can pass arrays in the usual array format, this can be wasteful as most elements are zero. For this reason, there are different ways to represent your matrix that are much more compact. These are given as classes in [scipy.sparse](https://docs.scipy.org/doc/scipy/reference/sparse.html).